In [4]:
!pip install duckdb
import pandas as pd
import duckdb

customerdata = pd.read_csv('CustomerData.csv')

print(customerdata.shape)

(6418, 32)


In [5]:
customerdata.head()

,Customer_ID,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,...,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason
0,19877-DEL,Male,35,No,Delhi,7,27,NaN,Yes,No,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,58353-MAH,Female,45,Yes,Maharashtra,14,13,NaN,Yes,Yes,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,25063-WES,Male,51,No,West Bengal,4,35,Deal 5,Yes,No,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,59787-KAR,Male,79,No,Karnataka,3,21,Deal 4,Yes,No,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,28544-TAM,Female,80,No,Tamil Nadu,3,8,NaN,Yes,No,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [6]:
# Connecting too DuckDb in-memory

con = duckdb.connect()

In [7]:
con.register('customerdata',customerdata)

In [8]:
# Checking the connection

print(con.execute("DESCRIBE customerdata").df())

                    column_name column_type null   key default extra
0                   Customer_ID     VARCHAR  YES  None    None  None
1                        Gender     VARCHAR  YES  None    None  None
2                           Age      BIGINT  YES  None    None  None
3                       Married     VARCHAR  YES  None    None  None
4                         State     VARCHAR  YES  None    None  None
5           Number_of_Referrals      BIGINT  YES  None    None  None
6              Tenure_in_Months      BIGINT  YES  None    None  None
7                    Value_Deal     VARCHAR  YES  None    None  None
8                 Phone_Service     VARCHAR  YES  None    None  None
9                Multiple_Lines     VARCHAR  YES  None    None  None
10             Internet_Service     VARCHAR  YES  None    None  None
11                Internet_Type     VARCHAR  YES  None    None  None
12              Online_Security     VARCHAR  YES  None    None  None
13                Online_Backup   

In [10]:
# Data Exploration - Checking Distinct Values by Gender

duckdb.query("""
SELECT Gender, Count(Gender) as TotalCount,
Count(Gender) * 100.0 / (Select Count(*) from customerdata)  as Percentage
from customerdata
Group by Gender;
""").df()

,Gender,TotalCount,Percentage
0,Female,4048,63.072608
1,Male,2370,36.927392


In [11]:
# Checking Distinct Values by Contract

duckdb.query("""
SELECT Contract, Count(Contract) as TotalCount,
Count(Contract) * 100.0 / (Select Count(*) from customerdata)  as Percentage
from customerdata
Group by Contract;
""").df()

,Contract,TotalCount,Percentage
0,Month-to-Month,3286,51.199751
1,Two Year,1719,26.784045
2,One Year,1413,22.016204


In [12]:
# Checking Distinct Values by Customer Status

duckdb.query("""
SELECT Customer_Status, Count(Customer_Status) as TotalCount, Sum(Total_Revenue) as TotalRev,
Sum(Total_Revenue) / (Select sum(Total_Revenue) from customerdata) * 100  as RevPercentage
from customerdata
Group by Customer_Status;
""").df()

,Customer_Status,TotalCount,TotalRev,RevPercentage
0,Joined,411,49281.56,0.253097
1,Stayed,4275,16010148.27,82.223960
2,Churned,1732,3411960.58,17.522943


In [14]:
# Checking Distinct Values by States

duckdb.query("""
SELECT State, Count(State) as TotalCount,
Count(State) * 1.0 / (Select Count(*) from customerdata)  as Percentage
from customerdata
Group by State
Order by Percentage desc;
""").df()

,State,TotalCount,Percentage
0,Uttar Pradesh,629,0.098006
1,Tamil Nadu,600,0.093487
2,Maharashtra,504,0.078529
3,Karnataka,470,0.073232
4,Haryana,398,0.062013
5,Andhra Pradesh,395,0.061546
6,West Bengal,368,0.057339
7,Punjab,342,0.053288
8,Bihar,336,0.052353
9,Gujarat,335,0.052197


In [15]:
# Checking null values in each column 

duckdb.query("""
SELECT 
    SUM(CASE WHEN Customer_ID IS NULL THEN 1 ELSE 0 END) AS Customer_ID_Null_Count,
    SUM(CASE WHEN Gender IS NULL THEN 1 ELSE 0 END) AS Gender_Null_Count,
    SUM(CASE WHEN Age IS NULL THEN 1 ELSE 0 END) AS Age_Null_Count,
    SUM(CASE WHEN Married IS NULL THEN 1 ELSE 0 END) AS Married_Null_Count,
    SUM(CASE WHEN State IS NULL THEN 1 ELSE 0 END) AS State_Null_Count,
    SUM(CASE WHEN Number_of_Referrals IS NULL THEN 1 ELSE 0 END) AS Number_of_Referrals_Null_Count,
    SUM(CASE WHEN Tenure_in_Months IS NULL THEN 1 ELSE 0 END) AS Tenure_in_Months_Null_Count,
    SUM(CASE WHEN Value_Deal IS NULL THEN 1 ELSE 0 END) AS Value_Deal_Null_Count,
    SUM(CASE WHEN Phone_Service IS NULL THEN 1 ELSE 0 END) AS Phone_Service_Null_Count,
    SUM(CASE WHEN Multiple_Lines IS NULL THEN 1 ELSE 0 END) AS Multiple_Lines_Null_Count,
    SUM(CASE WHEN Internet_Service IS NULL THEN 1 ELSE 0 END) AS Internet_Service_Null_Count,
    SUM(CASE WHEN Internet_Type IS NULL THEN 1 ELSE 0 END) AS Internet_Type_Null_Count,
    SUM(CASE WHEN Online_Security IS NULL THEN 1 ELSE 0 END) AS Online_Security_Null_Count,
    SUM(CASE WHEN Online_Backup IS NULL THEN 1 ELSE 0 END) AS Online_Backup_Null_Count,
    SUM(CASE WHEN Device_Protection_Plan IS NULL THEN 1 ELSE 0 END) AS Device_Protection_Plan_Null_Count,
    SUM(CASE WHEN Premium_Support IS NULL THEN 1 ELSE 0 END) AS Premium_Support_Null_Count,
    SUM(CASE WHEN Streaming_TV IS NULL THEN 1 ELSE 0 END) AS Streaming_TV_Null_Count,
    SUM(CASE WHEN Streaming_Movies IS NULL THEN 1 ELSE 0 END) AS Streaming_Movies_Null_Count,
    SUM(CASE WHEN Streaming_Music IS NULL THEN 1 ELSE 0 END) AS Streaming_Music_Null_Count,
    SUM(CASE WHEN Unlimited_Data IS NULL THEN 1 ELSE 0 END) AS Unlimited_Data_Null_Count,
    SUM(CASE WHEN Contract IS NULL THEN 1 ELSE 0 END) AS Contract_Null_Count,
    SUM(CASE WHEN Paperless_Billing IS NULL THEN 1 ELSE 0 END) AS Paperless_Billing_Null_Count,
    SUM(CASE WHEN Payment_Method IS NULL THEN 1 ELSE 0 END) AS Payment_Method_Null_Count,
    SUM(CASE WHEN Monthly_Charge IS NULL THEN 1 ELSE 0 END) AS Monthly_Charge_Null_Count,
    SUM(CASE WHEN Total_Charges IS NULL THEN 1 ELSE 0 END) AS Total_Charges_Null_Count,
    SUM(CASE WHEN Total_Refunds IS NULL THEN 1 ELSE 0 END) AS Total_Refunds_Null_Count,
    SUM(CASE WHEN Total_Extra_Data_Charges IS NULL THEN 1 ELSE 0 END) AS Total_Extra_Data_Charges_Null_Count,
    SUM(CASE WHEN Total_Long_Distance_Charges IS NULL THEN 1 ELSE 0 END) AS Total_Long_Distance_Charges_Null_Count,
    SUM(CASE WHEN Total_Revenue IS NULL THEN 1 ELSE 0 END) AS Total_Revenue_Null_Count,
    SUM(CASE WHEN Customer_Status IS NULL THEN 1 ELSE 0 END) AS Customer_Status_Null_Count,
    SUM(CASE WHEN Churn_Category IS NULL THEN 1 ELSE 0 END) AS Churn_Category_Null_Count,
    SUM(CASE WHEN Churn_Reason IS NULL THEN 1 ELSE 0 END) AS Churn_Reason_Null_Count
FROM customerdata;
""").df()

,Customer_ID_Null_Count,Gender_Null_Count,Age_Null_Count,Married_Null_Count,State_Null_Count,Number_of_Referrals_Null_Count,Tenure_in_Months_Null_Count,Value_Deal_Null_Count,Phone_Service_Null_Count,Multiple_Lines_Null_Count,...,Payment_Method_Null_Count,Monthly_Charge_Null_Count,Total_Charges_Null_Count,Total_Refunds_Null_Count,Total_Extra_Data_Charges_Null_Count,Total_Long_Distance_Charges_Null_Count,Total_Revenue_Null_Count,Customer_Status_Null_Count,Churn_Category_Null_Count,Churn_Reason_Null_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3548.0,0.0,622.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4686.0,4686.0


In [ ]:
# Removing Null Values and loading the data in a new table

CREATE OR REPLACE TABLE CHURN_CUSTOMERDATA AS
SELECT 
    Customer_ID,
    Gender,
    Age,
    Married,
    State,
    Number_of_Referrals,
    Tenure_in_Months,
    COALESCE(Value_Deal, 'None') AS Value_Deal,
    Phone_Service,
    COALESCE(Multiple_Lines, 'No') AS Multiple_Lines,
    Internet_Service,
    COALESCE(Internet_Type, 'None') AS Internet_Type,
    COALESCE(Online_Security, 'No') AS Online_Security,
    COALESCE(Online_Backup, 'No') AS Online_Backup,
    COALESCE(Device_Protection_Plan, 'No') AS Device_Protection_Plan,
    COALESCE(Premium_Support, 'No') AS Premium_Support,
    COALESCE(Streaming_TV, 'No') AS Streaming_TV,
    COALESCE(Streaming_Movies, 'No') AS Streaming_Movies,
    COALESCE(Streaming_Music, 'No') AS Streaming_Music,
    COALESCE(Unlimited_Data, 'No') AS Unlimited_Data,
    Contract,
    Paperless_Billing,
    Payment_Method,
    Monthly_Charge,
    Total_Charges,
    Total_Refunds,
    Total_Extra_Data_Charges,
    Total_Long_Distance_Charges,
    Total_Revenue,
    Customer_Status,
    COALESCE(Churn_Category, 'Others') AS Churn_Category,
    COALESCE(Churn_Reason, 'Others') AS Churn_Reason
FROM CUSTOMERDATA;

In [ ]:
# Creating view which can be later used in powerbi for building dashboards

Create or replace view V_ChurnCustomerData as (
select * from CHURN_CUSTOMERDATA where Customer_Status In ('Churned', 'Stayed')
);

In [ ]:
Create or replace view V_JoinCustomerData as (
select * from CHURN_CUSTOMERDATA where Customer_Status = 'Joined'
);